In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import scipy.io as scio
import numpy as np

In [3]:
model_path = '../2'

#Latent dimensions
latent_dim = 20

# Load mean pose
init_pose = scio.loadmat('../Data/mean_pose.mat')['mean_vector']
init_pose = np.transpose(init_pose,(1,0))
init_input = tf.tile(tf.expand_dims(init_pose, axis=0), [1, 1, 1])
print(init_input.shape)

(1, 1, 24)


2025-04-15 14:15:03.246954: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
indices = np.array([
    1261, 2701, 2776, 1914, 2443, 3422, 4252, 2911, 4184, 3280, 3007, 4,
    4323, 1821, 4369, 2310, 3539, 2442, 3928, 1604, 2736, 456, 3409, 1463,
    4356, 168, 1223, 2533, 1989, 1292, 1508, 3317, 2153, 2586, 798, 701,
    3094, 4636, 1138, 3695, 3548, 3531, 904, 2521, 1618, 1816, 825, 3453,
    1503, 748
])

In [6]:
text = np.load('../Data/text_script.npy')

#indices = np.random.randint(0, text.shape[0], size = 50)
print(indices)
text_description = text[indices]
print(text_description)


[1261 2701 2776 1914 2443 3422 4252 2911 4184 3280 3007    4 4323 1821
 4369 2310 3539 2442 3928 1604 2736  456 3409 1463 4356  168 1223 2533
 1989 1292 1508 3317 2153 2586  798  701 3094 4636 1138 3695 3548 3531
  904 2521 1618 1816  825 3453 1503  748]
['girl walking towards a guy' 'a woman making a speech'
 'one man is speaking to another' 'a man is describing how'
 'child on stage giving a performance'
 'a comedic performance is being done' 'one woman is singing and dancing'
 'this man probably a professor is teaching his students'
 'woman gives a ted talk while standing on a box'
 'a musical dance is going on' 'woman playing a guitar'
 'a chef in kitchen adding oil to a recipe'
 'the woman hold the cloth and talk to the girl'
 'it is a video of ayesha curry preparing some food in the kitchen'
 'a man checking a car' 'one person doing the nae nae dance'
 'one person dancing in front of the camera'
 'a lady preparing a dish with vegetables and explaining the recipe'
 'a woman is giv

In [5]:
case_number = 20

#Load the model
def load_encoder(model_path):
    #Output's dimension = 512
    encoder = hub.load(model_path)
    return encoder

encoder = load_encoder(model_path)
generator = tf.keras.models.load_model(f"../Hyperparameter_Tuning_and_Ablation_Study/Model/Case_{case_number}/LSTM_Test_Motion_Model_Epoch_150.npy")

In [8]:
min = -0.13500924408435822
max = 0.1370220184326172

# Normalize action data
def normalize_data(data, data_min, data_max):
    return 2 * ((data - data_min) / (data_max - data_min)) - 1

# Input: raw text
# Output: (time_seq, 24) 
def gen_t2a(test_script, init_pose):

    text_embeddings = encoder([test_script])
    test_script_normalize = normalize_data(text_embeddings, min, max)

    noise = tf.random.normal([1,latent_dim])
    generated_motion = generator.predict([noise,test_script_normalize,init_pose])

    return generated_motion

In [9]:
motion_list = []

for i in range(text_description.shape[0]):
    script = text_description[i]
    temp_list = []
    for j in range(20):
        generated_motion = gen_t2a(script, init_input)
        temp_list.append(generated_motion)
    motion_list.append(temp_list)

motion_array = np.array(motion_list)
print("Motion array shape:", motion_array.shape)
np.save(f'../Hyperparameter_Tuning_and_Ablation_Study/Model/Case_{case_number}/mmodality_motion.npy', motion_array)
print('Saved')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━

In [6]:
text = np.load('../Data/text_script.npy')

# Save the text to a file
with open('output_text.txt', 'w', encoding='utf-8') as file:
    for i in range(text.shape[0]):
        # Convert element to string and write to the file
        file.write(str(text[i]) + '\n')  # Adds a newline after each entry
